In [18]:
"""
ANÁLISIS DE HIPÓTESIS 1: ESTACIONALIDAD INVERSA
Matrimonios vs Divorcios en Guatemala (2011-2021)

Hipótesis: Los matrimonios alcanzan su pico en noviembre-diciembre (temporada festiva) 
mientras que los divorcios presentan su mínimo en estos mismos meses

VERSIÓN CORREGIDA - Maneja nombres de meses como strings
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("="*80)
print("ANÁLISIS DE HIPÓTESIS 1: ESTACIONALIDAD INVERSA")
print("Matrimonios vs Divorcios en Guatemala (2011-2021)")
print("="*80)

# ==============================================================================
# 1. CARGA Y PREPARACIÓN DE DATOS
# ==============================================================================

print("\n" + "="*80)
print("1. CARGA Y PREPARACIÓN DE DATOS")
print("="*80)

# Mapeos de conversión entre nombres y números de meses
MESES_A_NUMERO = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4,
    'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8,
    'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12,
    # Variaciones posibles
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
    'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
    'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}
MESES_NOMBRES = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 
    5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
    9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}

try:
    # Cargar datasets
    divorcios = pd.read_csv('div_full.csv', encoding='utf-8')
    matrimonios = pd.read_csv('mat_full.csv', encoding='utf-8')
    
    print(f"\n✓ Divorcios cargados: {divorcios.shape[0]:,} registros, {divorcios.shape[1]} variables")
    print(f"✓ Matrimonios cargados: {matrimonios.shape[0]:,} registros, {matrimonios.shape[1]} variables")
    
    # Verificar columnas disponibles
    print("\nColumnas en divorcios:")
    print(divorcios.columns.tolist()[:10])  # Mostrar primeras 10
    print("\nColumnas en matrimonios:")
    print(matrimonios.columns.tolist()[:10])  # Mostrar primeras 10
    
    # Verificar tipo de dato en MESOCU
    print(f"\nTipo de dato MESOCU en divorcios: {divorcios['MESOCU'].dtype}")
    print(f"Tipo de dato MESOCU en matrimonios: {matrimonios['MESOCU'].dtype}")
    print(f"\nEjemplos de valores en MESOCU (divorcios): {divorcios['MESOCU'].unique()[:5]}")
    print(f"Ejemplos de valores en MESOCU (matrimonios): {matrimonios['MESOCU'].unique()[:5]}")
    
except FileNotFoundError as e:
    print(f"\n❌ ERROR: No se encontraron los archivos CSV")
    print("Por favor, asegúrate de que 'div_full.csv' y 'mat_full.csv' estén en el directorio actual")
    exit()

# Normalizar columna de mes
def normalizar_mes(df, col_name='MESOCU'):
    """Convierte la columna de mes a número entero, manejando strings o números"""
    if df[col_name].dtype == 'object':  # Si son strings
        print(f"  → Convirtiendo {col_name} de strings a números...")
        df[col_name + '_NUM'] = df[col_name].map(MESES_A_NUMERO)
        # Verificar si hay valores no mapeados
        if df[col_name + '_NUM'].isna().any():
            print(f"    ⚠️ Advertencia: {df[col_name + '_NUM'].isna().sum()} valores no se pudieron convertir")
            print(f"    Valores únicos no mapeados: {df[df[col_name + '_NUM'].isna()][col_name].unique()}")
    else:  # Si ya son números
        print(f"  → {col_name} ya es numérico")
        df[col_name + '_NUM'] = df[col_name].astype(int)
    return df

print("\n📋 Normalizando columnas de mes...")
divorcios = normalizar_mes(divorcios)
matrimonios = normalizar_mes(matrimonios)

# ==============================================================================
# 2. ANÁLISIS DE SERIES TEMPORALES MENSUALES
# ==============================================================================

print("\n" + "="*80)
print("2. ANÁLISIS DE SERIES TEMPORALES MENSUALES")
print("="*80)

# Crear series temporales agregadas por mes de ocurrencia
divorcios_mensuales = divorcios.groupby('MESOCU_NUM').size().reset_index(name='Divorcios')
matrimonios_mensuales = matrimonios.groupby('MESOCU_NUM').size().reset_index(name='Matrimonios')

# Merge de ambos datasets
estacionalidad = pd.merge(divorcios_mensuales, matrimonios_mensuales, 
                          left_on='MESOCU_NUM', right_on='MESOCU_NUM', how='outer')

# Renombrar y ordenar
estacionalidad.rename(columns={'MESOCU_NUM': 'Mes'}, inplace=True)
estacionalidad = estacionalidad.sort_values('Mes').reset_index(drop=True)

# Añadir nombres de meses
estacionalidad['Mes_Nombre'] = estacionalidad['Mes'].map(MESES_NOMBRES)

# Calcular porcentajes
estacionalidad['Divorcios_Pct'] = (estacionalidad['Divorcios'] / estacionalidad['Divorcios'].sum()) * 100
estacionalidad['Matrimonios_Pct'] = (estacionalidad['Matrimonios'] / estacionalidad['Matrimonios'].sum()) * 100

# Normalizar para comparación (z-scores)
estacionalidad['Divorcios_Norm'] = (estacionalidad['Divorcios'] - estacionalidad['Divorcios'].mean()) / estacionalidad['Divorcios'].std()
estacionalidad['Matrimonios_Norm'] = (estacionalidad['Matrimonios'] - estacionalidad['Matrimonios'].mean()) / estacionalidad['Matrimonios'].std()

print("\n📊 ESTADÍSTICAS DESCRIPTIVAS POR MES")
print("-" * 80)
print(estacionalidad[['Mes_Nombre', 'Divorcios', 'Matrimonios', 'Divorcios_Pct', 'Matrimonios_Pct']].to_string(index=False))

# Identificar meses pico y valle
div_max = estacionalidad.loc[estacionalidad['Divorcios'].idxmax()]
div_min = estacionalidad.loc[estacionalidad['Divorcios'].idxmin()]
mat_max = estacionalidad.loc[estacionalidad['Matrimonios'].idxmax()]
mat_min = estacionalidad.loc[estacionalidad['Matrimonios'].idxmin()]

print("\n📈 MESES PICO Y VALLE")
print("-" * 80)
print(f"Divorcios - Pico: {div_max['Mes_Nombre']} ({div_max['Divorcios']:,} casos, {div_max['Divorcios_Pct']:.2f}%)")
print(f"Divorcios - Valle: {div_min['Mes_Nombre']} ({div_min['Divorcios']:,} casos, {div_min['Divorcios_Pct']:.2f}%)")
print(f"Matrimonios - Pico: {mat_max['Mes_Nombre']} ({mat_max['Matrimonios']:,} casos, {mat_max['Matrimonios_Pct']:.2f}%)")
print(f"Matrimonios - Valle: {mat_min['Mes_Nombre']} ({mat_min['Matrimonios']:,} casos, {mat_min['Matrimonios_Pct']:.2f}%)")

# Análisis específico de noviembre-diciembre
nov_dic = estacionalidad[estacionalidad['Mes'].isin([11, 12])]
print("\n🎄 ANÁLISIS TEMPORADA FESTIVA (NOVIEMBRE-DICIEMBRE)")
print("-" * 80)
print(nov_dic[['Mes_Nombre', 'Divorcios', 'Matrimonios', 'Divorcios_Pct', 'Matrimonios_Pct']].to_string(index=False))
print(f"\nTotal Nov-Dic - Divorcios: {nov_dic['Divorcios'].sum():,} ({nov_dic['Divorcios_Pct'].sum():.2f}%)")
print(f"Total Nov-Dic - Matrimonios: {nov_dic['Matrimonios'].sum():,} ({nov_dic['Matrimonios_Pct'].sum():.2f}%)")

# ==============================================================================
# 3. ANÁLISIS DE CORRELACIÓN
# ==============================================================================

print("\n" + "="*80)
print("3. ANÁLISIS DE CORRELACIÓN")
print("="*80)

# Correlación de Pearson (lineal)
pearson_r, pearson_p = pearsonr(estacionalidad['Divorcios'], estacionalidad['Matrimonios'])

# Correlación de Spearman (monotónica)
spearman_r, spearman_p = spearmanr(estacionalidad['Divorcios'], estacionalidad['Matrimonios'])

# Correlación con datos normalizados
pearson_norm_r, pearson_norm_p = pearsonr(estacionalidad['Divorcios_Norm'], estacionalidad['Matrimonios_Norm'])

print("\n📊 COEFICIENTES DE CORRELACIÓN")
print("-" * 80)
print(f"Pearson (datos originales):   r = {pearson_r:.4f}, p-value = {pearson_p:.6f}")
print(f"Spearman (ranking):           ρ = {spearman_r:.4f}, p-value = {spearman_p:.6f}")
print(f"Pearson (datos normalizados): r = {pearson_norm_r:.4f}, p-value = {pearson_norm_p:.6f}")

print("\n🔍 INTERPRETACIÓN DE LA CORRELACIÓN")
print("-" * 80)
if pearson_r < -0.5:
    correlacion_tipo = "FUERTE NEGATIVA"
elif pearson_r < -0.3:
    correlacion_tipo = "MODERADA NEGATIVA"
elif pearson_r < 0:
    correlacion_tipo = "DÉBIL NEGATIVA"
elif pearson_r < 0.3:
    correlacion_tipo = "DÉBIL POSITIVA"
elif pearson_r < 0.5:
    correlacion_tipo = "MODERADA POSITIVA"
else:
    correlacion_tipo = "FUERTE POSITIVA"

print(f"Correlación detectada: {correlacion_tipo}")

significancia = "SIGNIFICATIVA" if pearson_p < 0.05 else "NO SIGNIFICATIVA"
print(f"Significancia estadística (α=0.05): {significancia}")

if pearson_r < 0:
    print("\n✓ Se detectó correlación negativa, consistente con la hipótesis de estacionalidad inversa")
else:
    print("\n✗ Se detectó correlación positiva, contrario a la hipótesis de estacionalidad inversa")

# ==============================================================================
# 4. VISUALIZACIONES
# ==============================================================================

print("\n" + "="*80)
print("4. GENERANDO VISUALIZACIONES")
print("="*80)

# Crear figura con múltiples subplots
fig = plt.figure(figsize=(18, 12))

# 4.1 Series temporales absolutas superpuestas
ax1 = plt.subplot(3, 2, 1)
ax1_twin = ax1.twinx()

ax1.plot(estacionalidad['Mes_Nombre'], estacionalidad['Divorcios'], 
         marker='o', linewidth=2, markersize=8, color='#E74C3C', label='Divorcios')
ax1_twin.plot(estacionalidad['Mes_Nombre'], estacionalidad['Matrimonios'], 
              marker='s', linewidth=2, markersize=8, color='#3498DB', label='Matrimonios')

ax1.set_xlabel('Mes', fontsize=11, fontweight='bold')
ax1.set_ylabel('Número de Divorcios', fontsize=11, fontweight='bold', color='#E74C3C')
ax1_twin.set_ylabel('Número de Matrimonios', fontsize=11, fontweight='bold', color='#3498DB')
ax1.set_title('Serie Temporal: Divorcios vs Matrimonios (Valores Absolutos)', fontsize=12, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.tick_params(axis='y', labelcolor='#E74C3C')
ax1_twin.tick_params(axis='y', labelcolor='#3498DB')
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper left')
ax1_twin.legend(loc='upper right')

# 4.2 Series temporales normalizadas
ax2 = plt.subplot(3, 2, 2)
ax2.plot(estacionalidad['Mes_Nombre'], estacionalidad['Divorcios_Norm'], 
         marker='o', linewidth=2.5, markersize=8, color='#E74C3C', label='Divorcios (normalizado)')
ax2.plot(estacionalidad['Mes_Nombre'], estacionalidad['Matrimonios_Norm'], 
         marker='s', linewidth=2.5, markersize=8, color='#3498DB', label='Matrimonios (normalizado)')
ax2.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax2.set_xlabel('Mes', fontsize=11, fontweight='bold')
ax2.set_ylabel('Z-Score (Desviaciones Estándar)', fontsize=11, fontweight='bold')
ax2.set_title('Serie Temporal Normalizada: Comparación Directa', fontsize=12, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

# Resaltar noviembre y diciembre
nov_dic_indices = estacionalidad[estacionalidad['Mes'].isin([11, 12])].index
for idx in nov_dic_indices:
    ax2.axvspan(idx-0.4, idx+0.4, alpha=0.2, color='gold', label='Nov-Dic' if idx == nov_dic_indices[0] else '')

# 4.3 Diagrama de barras comparativo
ax3 = plt.subplot(3, 2, 3)
x = np.arange(len(estacionalidad))
width = 0.35

bars1 = ax3.bar(x - width/2, estacionalidad['Divorcios_Pct'], width, 
                label='Divorcios', color='#E74C3C', alpha=0.8, edgecolor='black')
bars2 = ax3.bar(x + width/2, estacionalidad['Matrimonios_Pct'], width, 
                label='Matrimonios', color='#3498DB', alpha=0.8, edgecolor='black')

ax3.set_xlabel('Mes', fontsize=11, fontweight='bold')
ax3.set_ylabel('Porcentaje del Total Anual (%)', fontsize=11, fontweight='bold')
ax3.set_title('Distribución Porcentual Mensual: Divorcios vs Matrimonios', fontsize=12, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels(estacionalidad['Mes_Nombre'], rotation=45, ha='right')
ax3.legend(loc='best', fontsize=10)
ax3.grid(True, alpha=0.3, axis='y')

# 4.4 Scatter plot con línea de tendencia
ax4 = plt.subplot(3, 2, 4)
# Usar números de mes para el color
scatter = ax4.scatter(estacionalidad['Divorcios'], estacionalidad['Matrimonios'], 
                     s=150, alpha=0.7, c=estacionalidad['Mes'], cmap='coolwarm', 
                     edgecolors='black', linewidth=1.5, vmin=1, vmax=12)

# Línea de regresión
z = np.polyfit(estacionalidad['Divorcios'], estacionalidad['Matrimonios'], 1)
p = np.poly1d(z)
ax4.plot(estacionalidad['Divorcios'], p(estacionalidad['Divorcios']), 
         "r--", linewidth=2, alpha=0.8, label=f'y = {z[0]:.2f}x + {z[1]:.2f}')

# Añadir etiquetas de meses
for idx, row in estacionalidad.iterrows():
    ax4.annotate(row['Mes_Nombre'][:3], 
                (row['Divorcios'], row['Matrimonios']),
                fontsize=8, ha='center', va='bottom')

ax4.set_xlabel('Número de Divorcios', fontsize=11, fontweight='bold')
ax4.set_ylabel('Número de Matrimonios', fontsize=11, fontweight='bold')
ax4.set_title(f'Correlación Divorcios-Matrimonios (r = {pearson_r:.3f})', fontsize=12, fontweight='bold')
ax4.legend(loc='best', fontsize=10)
ax4.grid(True, alpha=0.3)

# Colorbar
cbar = plt.colorbar(scatter, ax=ax4)
cbar.set_label('Mes del Año', fontsize=10)

# 4.5 Heatmap de correlación
ax5 = plt.subplot(3, 2, 5)
corr_matrix = estacionalidad[['Divorcios', 'Matrimonios', 'Divorcios_Norm', 'Matrimonios_Norm']].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='RdYlGn_r', center=0,
            square=True, linewidths=2, cbar_kws={"shrink": 0.8},
            ax=ax5, vmin=-1, vmax=1)
ax5.set_title('Matriz de Correlación', fontsize=12, fontweight='bold')

# 4.6 Análisis de diferencias mes a mes
ax6 = plt.subplot(3, 2, 6)
estacionalidad['Diferencia_Norm'] = estacionalidad['Matrimonios_Norm'] - estacionalidad['Divorcios_Norm']
colors = ['#2ECC71' if x > 0 else '#E74C3C' for x in estacionalidad['Diferencia_Norm']]
bars = ax6.bar(estacionalidad['Mes_Nombre'], estacionalidad['Diferencia_Norm'], 
               color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax6.axhline(y=0, color='black', linestyle='-', linewidth=2)
ax6.set_xlabel('Mes', fontsize=11, fontweight='bold')
ax6.set_ylabel('Diferencia (Mat. - Div.) en Z-Scores', fontsize=11, fontweight='bold')
ax6.set_title('Diferencia Normalizada: ¿Inversa?', fontsize=12, fontweight='bold')
ax6.tick_params(axis='x', rotation=45)
ax6.grid(True, alpha=0.3, axis='y')

# Añadir leyenda
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#2ECC71', label='Mat. > Div.'),
                   Patch(facecolor='#E74C3C', label='Div. > Mat.')]
ax6.legend(handles=legend_elements, loc='best', fontsize=9)

plt.tight_layout()
plt.savefig('hipotesis1_estacionalidad_completa.png', dpi=300, bbox_inches='tight')
print("✓ Gráfico completo guardado: hipotesis1_estacionalidad_completa.png")
plt.close()

# ==============================================================================
# 5. CLUSTERING: DETERMINACIÓN DEL NÚMERO ÓPTIMO DE CLUSTERS
# ==============================================================================

print("\n" + "="*80)
print("5. CLUSTERING: DETERMINACIÓN DEL NÚMERO ÓPTIMO")
print("="*80)

# Preparar datos para clustering
X = estacionalidad[['Divorcios', 'Matrimonios']].values

# Normalizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n📊 Datos preparados para clustering:")
print(f"   - Dimensiones: {X_scaled.shape}")
print(f"   - Features: Divorcios, Matrimonios (normalizados)")

# Método del codo y silhouette
inertias = []
silhouette_scores = []
K_range = range(2, 8)

print("\n🔍 Evaluando diferentes números de clusters...")
print("-" * 80)
print(f"{'K':<5} {'Inercia':<15} {'Silhouette':<15} {'Interpretación'}")
print("-" * 80)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_avg = silhouette_score(X_scaled, kmeans.labels_)
    silhouette_scores.append(silhouette_avg)
    
    if silhouette_avg > 0.7:
        interpretacion = "Excelente"
    elif silhouette_avg > 0.5:
        interpretacion = "Buena"
    elif silhouette_avg > 0.3:
        interpretacion = "Aceptable"
    else:
        interpretacion = "Pobre"
    
    print(f"{k:<5} {kmeans.inertia_:<15.2f} {silhouette_avg:<15.4f} {interpretacion}")

# Determinar mejor K
best_k = K_range[np.argmax(silhouette_scores)]
best_silhouette = max(silhouette_scores)

print("\n" + "="*80)
print(f"✓ MEJOR NÚMERO DE CLUSTERS: K = {best_k}")
print(f"  Silhouette Score: {best_silhouette:.4f}")
print("="*80)

# Visualizar método del codo y silhouette
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Método del codo
ax1.plot(K_range, inertias, marker='o', linewidth=2, markersize=10, color='#3498DB')
ax1.set_xlabel('Número de Clusters (K)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Inercia (Within-Cluster Sum of Squares)', fontsize=12, fontweight='bold')
ax1.set_title('Método del Codo', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(K_range)

# Silhouette scores
ax2.plot(K_range, silhouette_scores, marker='s', linewidth=2, markersize=10, color='#E74C3C')
ax2.axhline(y=0.5, color='green', linestyle='--', linewidth=2, alpha=0.5, label='Umbral Bueno (0.5)')
ax2.axvline(x=best_k, color='orange', linestyle='--', linewidth=2, alpha=0.7, label=f'Mejor K={best_k}')
ax2.set_xlabel('Número de Clusters (K)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Silhouette Score', fontsize=12, fontweight='bold')
ax2.set_title('Análisis de Silhouette', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(loc='best')
ax2.set_xticks(K_range)

plt.tight_layout()
plt.savefig('hipotesis1_elbow_silhouette.png', dpi=300, bbox_inches='tight')
print("\n✓ Gráfico de evaluación guardado: hipotesis1_elbow_silhouette.png")
plt.close()

# ==============================================================================
# 6. CLUSTERING CON K ÓPTIMO
# ==============================================================================

print("\n" + "="*80)
print(f"6. CLUSTERING CON K = {best_k}")
print("="*80)

# Aplicar clustering con mejor K
kmeans_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
estacionalidad['Cluster'] = kmeans_final.fit_predict(X_scaled)

print(f"\n✓ Clustering completado con {best_k} clusters")

# ==============================================================================
# 7. VERIFICACIÓN DE CALIDAD: MÉTODO DE LA SILUETA
# ==============================================================================

print("\n" + "="*80)
print("7. VERIFICACIÓN DE CALIDAD: ANÁLISIS DE SILUETA")
print("="*80)

# Calcular scores de silueta para cada muestra
silhouette_vals = silhouette_samples(X_scaled, estacionalidad['Cluster'])
silhouette_avg_final = silhouette_score(X_scaled, estacionalidad['Cluster'])

print(f"\n📊 Silhouette Score Promedio: {silhouette_avg_final:.4f}")

# Análisis por cluster
print("\n" + "-"*80)
print("Análisis de Silueta por Cluster:")
print("-"*80)
print(f"{'Cluster':<10} {'N Meses':<12} {'Silhouette Avg':<18} {'Calidad'}")
print("-"*80)

for cluster_id in range(best_k):
    cluster_silhouette_vals = silhouette_vals[estacionalidad['Cluster'] == cluster_id]
    cluster_size = len(cluster_silhouette_vals)
    cluster_avg = cluster_silhouette_vals.mean()
    
    if cluster_avg > 0.7:
        calidad = "Excelente"
    elif cluster_avg > 0.5:
        calidad = "Buena"
    elif cluster_avg > 0.3:
        calidad = "Aceptable"
    else:
        calidad = "Pobre"
    
    print(f"{cluster_id:<10} {cluster_size:<12} {cluster_avg:<18.4f} {calidad}")

# Visualización del diagrama de silueta
fig, ax = plt.subplots(figsize=(10, 6))

y_lower = 10
colors = plt.cm.Set3(np.linspace(0, 1, best_k))

for i in range(best_k):
    # Valores de silueta para el cluster i
    ith_cluster_silhouette_vals = silhouette_vals[estacionalidad['Cluster'] == i]
    ith_cluster_silhouette_vals.sort()
    
    size_cluster_i = ith_cluster_silhouette_vals.shape[0]
    y_upper = y_lower + size_cluster_i
    
    ax.fill_betweenx(np.arange(y_lower, y_upper),
                      0, ith_cluster_silhouette_vals,
                      facecolor=colors[i], edgecolor=colors[i], alpha=0.7)
    
    # Etiqueta del cluster
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, f'Cluster {i}')
    
    y_lower = y_upper + 10

ax.set_title(f'Diagrama de Silueta para {best_k} Clusters', fontsize=14, fontweight='bold')
ax.set_xlabel('Coeficiente de Silueta', fontsize=12, fontweight='bold')
ax.set_ylabel('Cluster', fontsize=12, fontweight='bold')

# Línea vertical para el score promedio
ax.axvline(x=silhouette_avg_final, color="red", linestyle="--", linewidth=2, 
           label=f'Promedio: {silhouette_avg_final:.3f}')
ax.legend(loc='best')
ax.set_xlim([-0.2, 1])
ax.set_ylim([0, len(X_scaled) + (best_k + 1) * 10])
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('hipotesis1_silhouette_diagram.png', dpi=300, bbox_inches='tight')
print("\n✓ Diagrama de silueta guardado: hipotesis1_silhouette_diagram.png")
plt.close()

# ==============================================================================
# 8. INTERPRETACIÓN DE CLUSTERS
# ==============================================================================

print("\n" + "="*80)
print("8. INTERPRETACIÓN DE CLUSTERS")
print("="*80)

# Análisis detallado por cluster
cluster_analysis = []

for cluster_id in range(best_k):
    cluster_data = estacionalidad[estacionalidad['Cluster'] == cluster_id]
    
    analysis = {
        'Cluster': cluster_id,
        'N_Meses': len(cluster_data),
        'Meses': ', '.join(cluster_data['Mes_Nombre'].tolist()),
        'Div_Promedio': cluster_data['Divorcios'].mean(),
        'Div_Std': cluster_data['Divorcios'].std(),
        'Mat_Promedio': cluster_data['Matrimonios'].mean(),
        'Mat_Std': cluster_data['Matrimonios'].std(),
        'Div_Pct_Total': cluster_data['Divorcios_Pct'].sum(),
        'Mat_Pct_Total': cluster_data['Matrimonios_Pct'].sum(),
        'Ratio_Mat_Div': cluster_data['Matrimonios'].mean() / cluster_data['Divorcios'].mean() if cluster_data['Divorcios'].mean() > 0 else 0
    }
    cluster_analysis.append(analysis)

# Convertir a DataFrame para mejor visualización
df_clusters = pd.DataFrame(cluster_analysis)

print("\n📊 RESUMEN ESTADÍSTICO POR CLUSTER")
print("-" * 80)
print(df_clusters[['Cluster', 'N_Meses', 'Div_Promedio', 'Mat_Promedio', 'Ratio_Mat_Div']].to_string(index=False))

print("\n" + "="*80)
print("ANÁLISIS DETALLADO POR CLUSTER")
print("="*80)

# Asignar nombres descriptivos a los clusters
cluster_names = {}
for i, row in df_clusters.iterrows():
    cluster_id = row['Cluster']
    ratio = row['Ratio_Mat_Div']
    div_pct = row['Div_Pct_Total']
    mat_pct = row['Mat_Pct_Total']
    
    # Lógica para nombrar clusters
    if ratio > 15:
        name = "🎉 TEMPORADA ALTA DE MATRIMONIOS"
    elif ratio < 13:
        name = "⚖️ TEMPORADA DE DIVORCIOS RELATIVOS"
    else:
        name = "📊 TEMPORADA EQUILIBRADA"
    
    cluster_names[cluster_id] = name

# Imprimir análisis detallado
for cluster_id in range(best_k):
    cluster_data = estacionalidad[estacionalidad['Cluster'] == cluster_id]
    
    print(f"\n{'='*80}")
    print(f"CLUSTER {cluster_id}: {cluster_names[cluster_id]}")
    print(f"{'='*80}")
    
    print(f"\n🗓️  Meses incluidos ({len(cluster_data)}):")
    print(f"   {', '.join(cluster_data['Mes_Nombre'].tolist())}")
    
    print(f"\n💔 Divorcios:")
    print(f"   Promedio: {cluster_data['Divorcios'].mean():.0f} casos/mes")
    print(f"   Rango: {cluster_data['Divorcios'].min():.0f} - {cluster_data['Divorcios'].max():.0f}")
    print(f"   % del total anual: {cluster_data['Divorcios_Pct'].sum():.2f}%")
    print(f"   Z-score promedio: {cluster_data['Divorcios_Norm'].mean():.3f}")
    
    print(f"\n💍 Matrimonios:")
    print(f"   Promedio: {cluster_data['Matrimonios'].mean():.0f} casos/mes")
    print(f"   Rango: {cluster_data['Matrimonios'].min():.0f} - {cluster_data['Matrimonios'].max():.0f}")
    print(f"   % del total anual: {cluster_data['Matrimonios_Pct'].sum():.2f}%")
    print(f"   Z-score promedio: {cluster_data['Matrimonios_Norm'].mean():.3f}")
    
    ratio = cluster_data['Matrimonios'].mean() / cluster_data['Divorcios'].mean()
    print(f"\n📈 Ratio Matrimonios/Divorcios: {ratio:.2f}:1")
    
    # Características especiales
    if 11 in cluster_data['Mes'].values or 12 in cluster_data['Mes'].values:
        print(f"\n🎄 CARACTERÍSTICAS ESPECIALES:")
        print(f"   ⭐ Incluye temporada festiva (Nov-Dic)")
        if cluster_data['Matrimonios_Norm'].mean() > 0.5:
            print(f"   ✓ Alta actividad matrimonial")
        if cluster_data['Divorcios_Norm'].mean() < -0.3:
            print(f"   ✓ Baja actividad de divorcios")

# Visualización de clusters
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 8.1 Scatter plot de clusters
ax1 = axes[0, 0]
for cluster_id in range(best_k):
    cluster_data = estacionalidad[estacionalidad['Cluster'] == cluster_id]
    ax1.scatter(cluster_data['Divorcios'], cluster_data['Matrimonios'],
               s=200, alpha=0.7, label=f'Cluster {cluster_id}: {cluster_names[cluster_id][:20]}...',
               edgecolors='black', linewidth=1.5)
    
    # Añadir etiquetas de meses
    for _, row in cluster_data.iterrows():
        ax1.annotate(row['Mes_Nombre'][:3], 
                    (row['Divorcios'], row['Matrimonios']),
                    fontsize=9, ha='center', va='center', fontweight='bold')

# Añadir centroides
centroides = scaler.inverse_transform(kmeans_final.cluster_centers_)
ax1.scatter(centroides[:, 0], centroides[:, 1], 
           s=400, marker='X', c='red', edgecolors='black', 
           linewidth=3, label='Centroides', zorder=10)

ax1.set_xlabel('Divorcios', fontsize=12, fontweight='bold')
ax1.set_ylabel('Matrimonios', fontsize=12, fontweight='bold')
ax1.set_title(f'Clusters de Estacionalidad (K={best_k})', fontsize=13, fontweight='bold')
ax1.legend(loc='best', fontsize=9)
ax1.grid(True, alpha=0.3)

# 8.2 Heatmap de características por cluster
ax2 = axes[0, 1]
cluster_features = estacionalidad.groupby('Cluster')[['Divorcios_Norm', 'Matrimonios_Norm']].mean()
sns.heatmap(cluster_features.T, annot=True, fmt='.2f', cmap='RdYlGn', 
            center=0, ax=ax2, cbar_kws={'label': 'Z-Score'},
            linewidths=2, square=True)
ax2.set_title('Perfil de Clusters (Normalizado)', fontsize=13, fontweight='bold')
ax2.set_xlabel('Cluster', fontsize=12, fontweight='bold')
ax2.set_ylabel('Variable', fontsize=12, fontweight='bold')

# 8.3 Distribución de meses por cluster
ax3 = axes[1, 0]
cluster_counts = estacionalidad['Cluster'].value_counts().sort_index()
colors_bar = plt.cm.Set3(np.linspace(0, 1, best_k))
bars = ax3.bar(cluster_counts.index, cluster_counts.values, color=colors_bar, 
               alpha=0.7, edgecolor='black', linewidth=1.5)
ax3.set_xlabel('Cluster', fontsize=12, fontweight='bold')
ax3.set_ylabel('Número de Meses', fontsize=12, fontweight='bold')
ax3.set_title('Distribución de Meses por Cluster', fontsize=13, fontweight='bold')
ax3.set_xticks(range(best_k))
ax3.grid(True, alpha=0.3, axis='y')

# Añadir valores en las barras
for bar in bars:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

# 8.4 Comparación de ratios por cluster
ax4 = axes[1, 1]
ratios = [df_clusters.loc[df_clusters['Cluster'] == i, 'Ratio_Mat_Div'].values[0] for i in range(best_k)]
bars = ax4.barh(range(best_k), ratios, color=colors_bar, alpha=0.7, 
                edgecolor='black', linewidth=1.5)
ax4.set_ylabel('Cluster', fontsize=12, fontweight='bold')
ax4.set_xlabel('Ratio Matrimonios/Divorcios', fontsize=12, fontweight='bold')
ax4.set_title('Intensidad Relativa por Cluster', fontsize=13, fontweight='bold')
ax4.set_yticks(range(best_k))
ax4.set_yticklabels([f'Cluster {i}' for i in range(best_k)])
ax4.grid(True, alpha=0.3, axis='x')

# Añadir valores
for i, (bar, ratio) in enumerate(zip(bars, ratios)):
    width = bar.get_width()
    ax4.text(width, bar.get_y() + bar.get_height()/2.,
            f'{ratio:.2f}:1',
            ha='left', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('hipotesis1_clusters_interpretacion.png', dpi=300, bbox_inches='tight')
print("\n✓ Visualización de clusters guardada: hipotesis1_clusters_interpretacion.png")
plt.close()

# ==============================================================================
# 9. VALIDACIÓN DE HIPÓTESIS
# ==============================================================================

print("\n" + "="*80)
print("9. VALIDACIÓN DE HIPÓTESIS")
print("="*80)

print("\n🔬 HIPÓTESIS PLANTEADA:")
print("-" * 80)
print("'Existe un patrón estacional opuesto entre matrimonios y divorcios:")
print(" los matrimonios alcanzan su pico en noviembre-diciembre (temporada festiva)")
print(" mientras que los divorcios presentan su mínimo en estos mismos meses'")

print("\n📊 EVIDENCIA ENCONTRADA:")
print("-" * 80)

# Criterio 1: Correlación negativa
print("\n1. CORRELACIÓN ENTRE VARIABLES:")
if pearson_r < 0:
    print(f"   ✓ Se encontró correlación NEGATIVA (r = {pearson_r:.4f})")
    print(f"   ✓ Significancia estadística: p = {pearson_p:.6f}")
    validacion_1 = True
else:
    print(f"   ✗ Se encontró correlación POSITIVA (r = {pearson_r:.4f})")
    print(f"   ✗ Contrario a la hipótesis de estacionalidad inversa")
    validacion_1 = False

# Criterio 2: Comportamiento en Nov-Dic
print("\n2. COMPORTAMIENTO EN TEMPORADA FESTIVA (NOV-DIC):")
nov_dic_mat = estacionalidad[estacionalidad['Mes'].isin([11, 12])]['Matrimonios'].sum()
nov_dic_div = estacionalidad[estacionalidad['Mes'].isin([11, 12])]['Divorcios'].sum()
nov_dic_mat_pct = (nov_dic_mat / estacionalidad['Matrimonios'].sum()) * 100
nov_dic_div_pct = (nov_dic_div / estacionalidad['Divorcios'].sum()) * 100

print(f"   Matrimonios Nov-Dic: {nov_dic_mat:,} ({nov_dic_mat_pct:.2f}% del total anual)")
print(f"   Divorcios Nov-Dic: {nov_dic_div:,} ({nov_dic_div_pct:.2f}% del total anual)")

if mat_max['Mes'] in [11, 12]:
    print(f"   ✓ Pico de matrimonios en {mat_max['Mes_Nombre']}")
    validacion_2a = True
else:
    print(f"   ✗ Pico de matrimonios en {mat_max['Mes_Nombre']} (no en Nov-Dic)")
    validacion_2a = False

if div_min['Mes'] in [11, 12]:
    print(f"   ✓ Mínimo de divorcios en {div_min['Mes_Nombre']}")
    validacion_2b = True
else:
    print(f"   ✗ Mínimo de divorcios en {div_min['Mes_Nombre']} (no en Nov-Dic)")
    validacion_2b = False

validacion_2 = validacion_2a and validacion_2b

# Criterio 3: Patrón en datos normalizados
print("\n3. ANÁLISIS DE SERIES NORMALIZADAS:")
nov_dic_norm = estacionalidad[estacionalidad['Mes'].isin([11, 12])]
mat_norm_avg = nov_dic_norm['Matrimonios_Norm'].mean()
div_norm_avg = nov_dic_norm['Divorcios_Norm'].mean()

print(f"   Z-score promedio de Matrimonios en Nov-Dic: {mat_norm_avg:.3f}")
print(f"   Z-score promedio de Divorcios en Nov-Dic: {div_norm_avg:.3f}")

if mat_norm_avg > 0.5 and div_norm_avg < -0.3:
    print(f"   ✓ Patrón inverso detectado: Matrimonios alto, Divorcios bajo")
    validacion_3 = True
else:
    print(f"   ✗ No se detecta patrón inverso claro en la temporada festiva")
    validacion_3 = False

# Criterio 4: Clustering revela grupos opuestos
print("\n4. AGRUPAMIENTO Y PATRONES:")
print(f"   Número óptimo de clusters: {best_k}")
print(f"   Calidad del clustering (Silhouette): {silhouette_avg_final:.4f}")

# Verificar si hay cluster que contenga Nov-Dic con características esperadas
cluster_festivo = estacionalidad[estacionalidad['Mes'].isin([11, 12])]['Cluster'].mode()[0]
cluster_festivo_data = estacionalidad[estacionalidad['Cluster'] == cluster_festivo]

if cluster_festivo_data['Matrimonios_Norm'].mean() > 0 and cluster_festivo_data['Divorcios_Norm'].mean() < 0:
    print(f"   ✓ Cluster {cluster_festivo} muestra patrón inverso en temporada festiva")
    validacion_4 = True
else:
    print(f"   ✗ Cluster {cluster_festivo} no muestra patrón inverso claro")
    validacion_4 = False

# CONCLUSIÓN FINAL
print("\n" + "="*80)
print("CONCLUSIÓN FINAL")
print("="*80)

validaciones = [validacion_1, validacion_2, validacion_3, validacion_4]
porcentaje_validacion = (sum(validaciones) / len(validaciones)) * 100

print(f"\nCriterios validados: {sum(validaciones)}/{len(validaciones)} ({porcentaje_validacion:.1f}%)")

if porcentaje_validacion >= 75:
    conclusion = "✓ HIPÓTESIS CONFIRMADA"
    print(f"\n{conclusion}")
    print("La evidencia estadística RESPALDA la hipótesis de estacionalidad inversa.")
    print("Se observa un patrón donde los matrimonios aumentan significativamente en")
    print("la temporada festiva (Nov-Dic) mientras que los divorcios disminuyen.")
elif porcentaje_validacion >= 50:
    conclusion = "⚠️ HIPÓTESIS PARCIALMENTE CONFIRMADA"
    print(f"\n{conclusion}")
    print("La evidencia muestra APOYO PARCIAL a la hipótesis de estacionalidad inversa.")
    print("Algunos criterios se cumplen, pero otros requieren mayor investigación.")
else:
    conclusion = "✗ HIPÓTESIS REFUTADA"
    print(f"\n{conclusion}")
    print("La evidencia NO respalda la hipótesis de estacionalidad inversa.")
    print("Los datos sugieren patrones estacionales diferentes a los esperados.")

# ==============================================================================
# 10. EXPORTAR RESULTADOS
# ==============================================================================

print("\n" + "="*80)
print("10. EXPORTANDO RESULTADOS")
print("="*80)

# Guardar tabla de estacionalidad
estacionalidad_export = estacionalidad.copy()
estacionalidad_export['Cluster_Nombre'] = estacionalidad_export['Cluster'].map(cluster_names)
estacionalidad_export.to_csv('hipotesis1_estacionalidad_resultados.csv', index=False)
print("✓ Tabla de estacionalidad guardada: hipotesis1_estacionalidad_resultados.csv")

# Guardar resumen de clusters
df_clusters['Nombre'] = df_clusters['Cluster'].map(cluster_names)
df_clusters.to_csv('hipotesis1_clusters_resumen.csv', index=False)
print("✓ Resumen de clusters guardado: hipotesis1_clusters_resumen.csv")

# Crear reporte de validación
reporte = {
    'Hipotesis': ['Estacionalidad Inversa'],
    'Correlacion_Pearson': [pearson_r],
    'P_Value': [pearson_p],
    'Silhouette_Score': [silhouette_avg_final],
    'Mejor_K': [best_k],
    'Porcentaje_Validacion': [porcentaje_validacion],
    'Conclusion': [conclusion]
}
df_reporte = pd.DataFrame(reporte)
df_reporte.to_csv('hipotesis1_reporte_validacion.csv', index=False)
print("✓ Reporte de validación guardado: hipotesis1_reporte_validacion.csv")

print("\n" + "="*80)
print("✓ ANÁLISIS COMPLETADO")
print("="*80)
print("\nArchivos generados:")
print("  1. hipotesis1_estacionalidad_completa.png - Visualizaciones principales")
print("  2. hipotesis1_elbow_silhouette.png - Evaluación de K óptimo")
print("  3. hipotesis1_silhouette_diagram.png - Diagrama de silueta")
print("  4. hipotesis1_clusters_interpretacion.png - Interpretación de clusters")
print("  5. hipotesis1_estacionalidad_resultados.csv - Datos procesados")
print("  6. hipotesis1_clusters_resumen.csv - Resumen estadístico de clusters")
print("  7. hipotesis1_reporte_validacion.csv - Resumen de validación")
print("\n" + "="*80)

ANÁLISIS DE HIPÓTESIS 1: ESTACIONALIDAD INVERSA
Matrimonios vs Divorcios en Guatemala (2011-2021)

1. CARGA Y PREPARACIÓN DE DATOS

✓ Divorcios cargados: 56,349 registros, 12 variables
✓ Matrimonios cargados: 752,264 registros, 15 variables

Columnas en divorcios:
['AÑOREG', 'DEPOCU', 'DEPREG', 'DIAOCU', 'EDADHOM', 'EDADMUJ', 'MESOCU', 'MESREG', 'MUPOCU', 'MUPREG']

Columnas en matrimonios:
['AÑOREG', 'CLAUNI', 'DEPOCU', 'DEPREG', 'DIAOCU', 'EDADHOM', 'EDADMUJ', 'ESCHOM', 'ESCMUJ', 'MESOCU']

Tipo de dato MESOCU en divorcios: object
Tipo de dato MESOCU en matrimonios: object

Ejemplos de valores en MESOCU (divorcios): ['Febrero' 'Abril' 'Septiembre' 'Julio' 'Diciembre']
Ejemplos de valores en MESOCU (matrimonios): ['Septiembre' 'Noviembre' 'Diciembre' 'Octubre' 'Julio']

📋 Normalizando columnas de mes...
  → Convirtiendo MESOCU de strings a números...
  → Convirtiendo MESOCU de strings a números...

2. ANÁLISIS DE SERIES TEMPORALES MENSUALES

📊 ESTADÍSTICAS DESCRIPTIVAS POR MES
-------